In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 5.1 MB/s 
     |████████████████████████████████| 216 kB 66.5 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=a53a9be978ebb6862e8dbbbd3a726690838187570ee25a50c97c188fb25b7f14
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


In [7]:
import os
os.environ["CORENLP_HOME"] = '/content/drive/MyDrive/project/stanford-corenlp-4.5.0'
import pandas


In [8]:
import numpy as np

In [9]:
import stanza
properties1 = {
    'openie.affinity_probability_cap': 2/3,
    'openie.triple.strict': 'true',
    'openie.resolve_coref': 'true',
    'max_entailments_per_clause': 5000,
}
# Import client module
from stanza.server import CoreNLPClient

client = CoreNLPClient(timeout=150000000, be_quiet=True, 
                       annotators=['openie'], 
endpoint='http://localhost:9012',properties=properties1)
client.start()
import time
time.sleep(10)

INFO:stanza:Writing properties to tmp file: corenlp_server-6e3811ece09a469a.props
INFO:stanza:Starting server with command: java -Xmx5G -cp /content/drive/MyDrive/project/stanford-corenlp-4.5.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9012 -timeout 150000000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-6e3811ece09a469a.props -annotators openie -preload -outputFormat serialized


In [10]:
csv_sentences = pandas.read_csv('/content/drive/MyDrive/project/openbook.txt', sep="\t", header=None)
csv_sentences.columns = ["sentence"]

In [11]:
csv_sentences

,sentence
0,A bee is a pollinating animal
1,A bird is a pollinating animal
2,An electrical conductor is a vehicle for the f...
3,An example of a change in the Earth is an ocea...
4,An example of a chemical change is acid breaki...
...,...
1321,wood is an electrical energy insulator
1322,wood is an thermal energy insulator
1323,xylem carries water from the roots of a plant ...
1324,xylem transports materials through the plant


In [12]:
text_list=csv_sentences["sentence"]

In [13]:
import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(u"glaciers melting has a negative impact on the glaicial environment")
# words = [word.lemma_ for word in text.split()]
# text = " ".join(words)
lemma=[]

for token in doc:
    lemma.append(token.lemma_)
text = " ".join(lemma)

In [14]:
### produce files
predicates=[]
# %的情况还没处理 percent 不记得两边有没有_
# :的情况还未处理 _dot_
# .的情况还未处理 _dot_
# be_from(moon,only_about_238,000_mile). weigh_around(car,number_5,000_lbs). have(apex,elevation_great_than_35000_ft).be_in(number_1,440_minute,one_day). 形成三个predicate，暂时删掉逗号
for text in text_list:
  text = text.lower()
  neg=False
  if text.find("not")!=-1 or text.find("n\'t")!=-1:
    neg=True
    text=text.replace("not "," ")
    text=text.replace("n't "," ")
  # print(neg)
  # A being may have more of a chance of survival if it can climb up to tall places
  # Albert Einstein was a German-born theoretical physicist. He developed the theory of relativity.
  # You can make a telescope with a starw
  # glaciers melting has a negative impact on the glaicial environment
  # Using mirrors to focus collected light from heavenly bodies allows detailed observation    error
  # properties1 = {
  #     'openie.affinity_probability_cap': 2/3,
  #     'openie.triple.strict': 'true',
  #     'openie.resolve_coref': 'true',
  #     # 'max_entailments_per_clause': 1,
  # }
  document = client.annotate(text,properties={"annotators":"tokenize,ssplit,lemma,pos,depparse,natlog,ner,coref,openie",
                                 "outputFormat": "json","triple.strict":"true","resolve_coref":"true"})
  triples = []
  for sentence in document['sentences']:
      for triple in sentence['openie']:
         triples.append({
            'subject': triple['subject'],
            'relation': triple['relation'],
              'object': triple['object']
          })
  # print(triples)


  # be(sun,plant)
  # allow(using, detailed observation)
  # Poison causes harm to (A) a Tree (B) a robot (C) a house (D) a car
  # poison causes harm to living things, plants are living things, Trees are plants, robots are not living things
  # pick 
  triple_list=[]
  triple_list.append(triples)
  # print(triple_list)
  # print(len(triple_list))
  # print(len(triples))
  for i in range(len(triple_list)):
    for j in range(len(triple_list[i])):
      # print(triple_list[i][j])
      # temp_triple=triple_list[i][j].copy()
      # temp_triple['subject']='asd'
      # print(temp_triple,' temp')
      # print(triple_list[i][j])
      subject=triple_list[i][j]['subject']
      subject_doc=nlp(subject)
      temp1=[]
      for token in subject_doc:
        temp1.append(token.lemma_)
        subject_lemma = " ".join(temp1)
      if subject_lemma[0].isdigit():
        subject_lemma="number_"+subject_lemma
      # print(subject_lemma,'******')
    
      relation=triple_list[i][j]['relation']
      relation_doc=nlp(relation)
      temp2=[]
      for token in relation_doc:
        temp2.append(token.lemma_)
        relation_lemma = " ".join(temp2)
      if relation_lemma[0].isdigit():
        relation_lemma="number_"+relation_lemma
      # print(relation_lemma,'******')

      obj=triple_list[i][j]['object']
      obj_doc=nlp(obj)
      temp3=[]
      for token in obj_doc:
        temp3.append(token.lemma_)
        obj_lemma = " ".join(temp3)
      if obj_lemma[0].isdigit():
        obj_lemma="number_"+obj_lemma
      # print(obj_lemma,'******')

      # print(subject,'+',relation,'+',obj)
      temp_predicate=relation_lemma+'('+subject_lemma+','+obj_lemma+')'
      # print("lemma",temp_predicate.le)
      temp_predicate=temp_predicate.replace(" ","_")
      temp_predicate=temp_predicate.lower()
      # print(temp_predicate)
      if neg==False:
        predicates.append(temp_predicate)
      if neg==True:
        neg_temp_predicate='not '+ temp_predicate
        predicates.append(neg_temp_predicate)
print(predicates)


['be(bee,animal)', 'be(bee,pollinate_animal)', 'be(bee,pollinate)', 'be(bird,animal)', 'be(bird,pollinate)', 'be(bird,pollinate_animal)', 'be_vehicle_for(conductor,flow_of_electricity)', 'be_vehicle_for(conductor,flow)', 'be(electrical_conductor,vehicle)', 'be_vehicle_for(electrical_conductor,flow_of_electricity)', 'be_vehicle_for(electrical_conductor,flow)', 'be(conductor,vehicle)', 'be(example,ocean_become)', 'example_of_be(change,ocean)', 'be_in(change,earth)', 'be(example,ocean)', 'example_of_be(change,ocean_become)', 'be_break_down(example,substance)', 'example_of(substance,chemical_change)', 'be(example,footprint)', 'be_footprint_in(example,rock)', 'example_of_be(fossil,footprint)', 'be_in(footprint,rock)', 'example_of_be(fossil,footprint_in_rock)', 'be(example,paw_print)', 'example_of_be(fossil,paw_print_in_rock)', 'example_of_be(fossil,paw_print)', 'be_in(paw_print,rock)', 'be(example,paw_print_in_rock)', 'be(example,bone)', 'example_of_be(fossil,bone)', 'be_bone_of(example,ext

In [15]:
print(len(predicates))

5474


In [16]:
csv_sentences = pandas.read_csv('/content/drive/MyDrive/project/crowdsourced-facts.txt', sep="\t", header=None)
csv_sentences.columns = ["sentence"]

In [17]:
csv_sentences

,sentence
0,-2+2=0
1,1 day has 1440 minutes
2,1 out of 4 people in the world are chronically...
3,100 degrees celsius is the boiling point of water
4,12:00 PM is noon
...,...
5170,zero is nothing
5171,zipping powered is electric
5172,zooplankton do not have respiration systems
5173,zoos are public exhibitions for viewing


In [18]:
text_list=csv_sentences["sentence"]

In [19]:
### produce files
predicates1=[]
# %的情况还没处理 percent 不记得两边有没有_
# :的情况还未处理 _dot_
# .的情况还未处理 _dot_
# be_from(moon,only_about_238,000_mile). weigh_around(car,number_5,000_lbs). have(apex,elevation_great_than_35000_ft).be_in(number_1,440_minute,one_day). 形成三个predicate，暂时删掉逗号
for text in text_list:
  text = text.lower()
  neg=False
  if text.find("not")!=-1 or text.find("n\'t")!=-1:
    neg=True
    text=text.replace("not "," ")
    text=text.replace("n't "," ")
  # print(neg)
  # A being may have more of a chance of survival if it can climb up to tall places
  # Albert Einstein was a German-born theoretical physicist. He developed the theory of relativity.
  # You can make a telescope with a starw
  # glaciers melting has a negative impact on the glaicial environment
  # Using mirrors to focus collected light from heavenly bodies allows detailed observation    error
  # properties1 = {
  #     'openie.affinity_probability_cap': 2/3,
  #     'openie.triple.strict': 'true',
  #     'openie.resolve_coref': 'true',
  #     # 'max_entailments_per_clause': 1,
  # }
  document = client.annotate(text,properties={"annotators":"tokenize,ssplit,lemma,pos,depparse,natlog,ner,coref,openie",
                                 "outputFormat": "json","triple.strict":"true","resolve_coref":"true"})
  triples = []
  for sentence in document['sentences']:
      for triple in sentence['openie']:
         triples.append({
            'subject': triple['subject'],
            'relation': triple['relation'],
              'object': triple['object']
          })
  # print(triples)


  # be(sun,plant)
  # allow(using, detailed observation)
  # Poison causes harm to (A) a Tree (B) a robot (C) a house (D) a car
  # poison causes harm to living things, plants are living things, Trees are plants, robots are not living things
  # pick 
  triple_list=[]
  triple_list.append(triples)
  # print(triple_list)
  # print(len(triple_list))
  # print(len(triples))
  for i in range(len(triple_list)):
    for j in range(len(triple_list[i])):
      # print(triple_list[i][j])
      # temp_triple=triple_list[i][j].copy()
      # temp_triple['subject']='asd'
      # print(temp_triple,' temp')
      # print(triple_list[i][j])
      subject=triple_list[i][j]['subject']
      subject_doc=nlp(subject)
      temp1=[]
      for token in subject_doc:
        temp1.append(token.lemma_)
        subject_lemma = " ".join(temp1)
      if subject_lemma[0].isdigit():
        subject_lemma="number_"+subject_lemma
      # print(subject_lemma,'******')
    
      relation=triple_list[i][j]['relation']
      relation_doc=nlp(relation)
      temp2=[]
      for token in relation_doc:
        temp2.append(token.lemma_)
        relation_lemma = " ".join(temp2)
      if relation_lemma[0].isdigit():
        relation_lemma="number_"+relation_lemma
      # print(relation_lemma,'******')

      obj=triple_list[i][j]['object']
      obj_doc=nlp(obj)
      temp3=[]
      for token in obj_doc:
        temp3.append(token.lemma_)
        obj_lemma = " ".join(temp3)
      if obj_lemma[0].isdigit():
        obj_lemma="number_"+obj_lemma
      # print(obj_lemma,'******')

      # print(subject,'+',relation,'+',obj)
      temp_predicate=relation_lemma+'('+subject_lemma+','+obj_lemma+')'
      # print("lemma",temp_predicate.le)
      temp_predicate=temp_predicate.replace(" ","_")
      temp_predicate=temp_predicate.lower()
      # print(temp_predicate)
      if neg==False:
        predicates1.append(temp_predicate)
      if neg==True:
        neg_temp_predicate='not '+ temp_predicate
        predicates1.append(neg_temp_predicate)
print(predicates1)

['have(day,number_1440_minute)', 'have(number_1_day,number_1440_minute)', 'be_in(number_1_4_people,world)', 'be(degree_celsius,boiling_point_of_water)', 'be(degree_celsius,boiling_point)', 'be(number_100_degree_celsius,boiling_point)', 'be(number_100_degree_celsius,boiling_point_of_water)', 'be(number_12:00_pm,noon)', 'be(pm,noon)', 'be(number_168_hour,week)', 'be(hour,week)', 'be_close_distance_of(number_4_foot,number_4_option)', 'be(number_4_foot,distance)', 'be(number_4_foot,close_distance)', 'be(foot,distance)', 'be(foot,close_distance)', 'be_distance_of(foot,number_4_option)', 'be_close_distance_of(foot,number_4_option)', 'be_distance_of(number_4_foot,number_4_option)', 'be(degree_celsius,coldest)', 'be_cold_of(degree_celsius,four_temperature)', 'be_cold_of(number_5_degree_celsius,four_temperature)', 'be(number_5_degree_celsius,coldest)', 'be_cold_of(number_5_degree_celsius,four_give_temperature)', 'be_cold_of(degree_celsius,four_give_temperature)', 'be(number_500_ml,measure)', 'b

In [20]:
print(len(predicates1))

12828


In [21]:

#要在后面加上点号.还未实现
#有一个predicate是'的情况还未处理
predicate_file=open('/content/drive/MyDrive/project/predicate.txt',mode='w')
for predicate in predicates:
  predicate_file.write(predicate)
  predicate_file.write('\n')
for predicate in predicates1:
  predicate_file.write(predicate)
  predicate_file.write('\n')
predicate_file.close()